In [ ]:
#1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/MyDrive/ColabFiles/houseDataset2024.csv"  /content/houseDataset2024.csv
!cp -R "/content/drive/MyDrive/ColabFiles/client" /content/client

ValueError: mount failed

In [ ]:
#2
df=pd.read_csv("houseDataset2024.csv")

In [ ]:
#3
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.info()

In [ ]:
#4
df=df[df["type"]=="House"]
df.drop(columns=["url","title","area","type","area_sqm","price_per_sqm","date_added","Latitude","Longitude"],inplace=True)
df.info()

In [ ]:
#5
df.groupby("city")["city"].agg('count')

In [ ]:
#6
df['bedrooms'] = df['bedrooms'].str.extract('(\d+)').astype(int)
df['baths'] = df['baths'].str.extract('(\d+)').astype(int)

In [ ]:
# Calculate the correlation matrix
numeric_cols = df.select_dtypes(include=['float64', 'int64'])
corr_matrix = numeric_cols.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix of Numeric Variables')
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
#7
df["city"]=pd.factorize(df["city"])[0]

In [ ]:
df.tail()

In [ ]:
df["bedrooms"].unique()

In [ ]:
df[df.bedrooms>=6]

In [ ]:
df["area_sqft"].unique()

In [ ]:
address_stats=df.groupby("address")["address"].agg('count').sort_values(ascending=False)
len(df.address.unique())
df.shape

In [ ]:
len(address_stats[address_stats<=10])

In [ ]:
address_less_than_10 = address_stats[address_stats<=10]
address_less_than_10

In [ ]:
# handle outliers
df.address=df.address.apply(lambda x: "other" if x in address_less_than_10 else x)
len(df.address.unique())


In [ ]:
# df=df.drop(df.index[47950])
# df.tail()
# df.shape

In [ ]:
df1=df[~(df.area_sqft/df.bedrooms<300)]
df1=df1[~df['price_per_sqft'].isin([float('inf'), float('-inf')])]
df1.shape

In [ ]:
df1.info()

In [ ]:
df1.head()

In [ ]:
df.price.describe()

In [ ]:
def remove_pps_outliers(df):
  df_out=pd.DataFrame()
  for key,subdf in df.groupby("address"):
    m=np.mean(subdf.price_per_sqft)
    st=np.std(subdf.price_per_sqft)
    reduced_df=subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
    df_out=pd.concat([df_out,reduced_df],ignore_index=True)
  return df_out

df2=remove_pps_outliers(df1)
df2.shape

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
df2=df2[df2.bedrooms!=0]
df2=df2[df2.baths!=0]
df2.shape

In [ ]:
df2.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

In [ ]:
df2.shape

In [ ]:
df2.columns = df2.columns.str.lower()
df2['address'] = df2['address'].str.lower()
dummies=pd.get_dummies(df2.address)
dummies.head()


In [ ]:
df2.info()

In [ ]:
df3=pd.concat([df2,dummies],axis='columns')

df3.head()

In [ ]:
df3.drop(columns=['address','price_per_sqft','city'],inplace=True)
df3.head()

In [ ]:
df3.to_csv('data1.csv')
!cp data.csv "drive/My Drive/"

In [ ]:
# threshold = df3['price'].quantile(0.90)
# df3 = df3[df3['price'] <= threshold]
# df3.shape

In [ ]:
x=df3.drop('price',axis='columns')
x.head()

In [ ]:
y=df3.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_train, y_train)
print(model.score(x_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# # Train model
# rf = RandomForestRegressor(n_estimators=100, random_state=42)
# rf.fit(x_train, y_train)

# # Predictions
# y_train_pred = rf.predict(x_train)
# y_test_pred = rf.predict(x_test)

# Evaluate model
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")


In [ ]:
x.columns

In [ ]:
def predict_price(location,bedroom,bath,sqft):
  loc_index=np.where(x.columns==location)[0][0]
  X=np.zeros(len(x.columns))
  X[0]=bedroom
  X[1]=bath
  X[2]=sqft
  if loc_index>=0:
    X[loc_index]=1
  return model.predict([X])[0]

In [ ]:
predict_price("wazir town, rawalpindi, punjab",5,3,1500 )

In [ ]:
import pickle
with open("real_estate_model.pickle","wb")as f:
  pickle.dump(model,f)

In [ ]:
# from google.colab import files
# files.download('real_estate_model.pickle')


In [ ]:
import json
columns={
    'data_columns': [col.lower() for col in x.columns]
}
with open ("columns.json","w")as f:
  f.write(json.dumps(columns))

In [ ]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok
!pip install pyngrok
!ngrok authtoken "2lCxGRma31qgLuD18mSmBYdkPxm_5D4H1PK1fex6AnoGYc7kA"

In [ ]:
import pickle
import json
import numpy as np
from pyngrok import ngrok



__locations = None
__data_columns = None
__model = None

def load_saved_artifacts():
    print("loading saved artifacts...start")
    global  __data_columns
    global __locations

    with open("columns.json", "r") as f:
        __data_columns = json.load(f)['data_columns']
        __locations = __data_columns[3:]

    global __model
    if __model is None:
        with open('real_estate_model.pickle', 'rb') as f:

            __model = pickle.load(f)
    print("loading saved artifacts...done")

def get_location_names():
    print("Locations loaded:", __locations)
    return __locations

def get_data_columns():
    return __data_columns


if __name__ == '__main__':
    load_saved_artifacts()
    # print(get_location_names())
    # print(get_estimated_price("bahria town, lahore, punjab",2500,4,4))


In [ ]:
!pip install flask_ngrok
from flask import Flask, request, jsonify
from flask import send_from_directory
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# run_with_ngrok(app)
@app.route('/',methods=['GET'])
def open_home_page():
  return send_from_directory("/content/client/", "app.html")

@app.route('/<path:filename>')
def serve_static_file(filename):
    return send_from_directory("/content/client/", filename)


@app.route('/get_location_names',methods=['GET'])
def get_location_names_route():
    response = jsonify({
        'locations': get_location_names()
    })
    response.headers.add('Access-Control-Allow-Origin', '*')

    return response

@app.route('/predict_home_price', methods=['GET','POST'])
def predict_home_price():
    total_sqft = float(request.form['total_sqft'])
    location = request.form['location']
    bedroom = int(request.form['bedroom'])
    bath = int(request.form['bath'])


    response = jsonify({
        'estimated_price': predict_price(location,bedroom,bath,total_sqft)
    })

    response.headers.add('Access-Control-Allow-Origin', '*')

    return response


# @app.route('/distribute_inheritance', methods=['POST'])
# def calculate():
#     house_price = int(request.form['InheritancePrice'])
#     sons = int(request.form['sons'])
#     daughters = int(request.form['daughters'])
#     sisters = int(request.form['sisters'])
#     brothers = int(request.form['brothers'])
#     wives = int(request.form['wives'])
#     husband = request.form['husband'] == "yes"
#     mother = request.form['mother'] == "yes"
#     father = request.form['father'] == "yes"

#     response = jsonify({
#         'Inheritance': util.distribute_inheritance(house_price,sons,daughters,brothers,sisters,husband,wives,father,mother)
#     })
#     response.headers.add('Access-Control-Allow-Origin', '*')

#     return response

if __name__ == "__main__":
    print("Starting Python Flask Server For Home Price Prediction...")
    load_saved_artifacts()
    app.run()

 * ngrok tunnel "https://1f3f-34-80-147-247.ngrok-free.app" -> "http://127.0.0.1:5000"
Starting Python Flask Server For Home Price Prediction...
loading saved artifacts...start
loading saved artifacts...done
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:42] "GET /app.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:42] "GET /app.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:43] "GET /get_location_names HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:43] "GET /favicon.ico HTTP/1.1" 404 -


Locations loaded: ['abdalians cooperative housing society, lahore, punjab', 'adiala road, rawalpindi, punjab', 'afshan colony, rawalpindi, punjab', 'airport enclave, islamabad, islamabad capital', 'airport housing society, rawalpindi, punjab', 'airport road, lahore, punjab', 'al jalil garden, lahore, punjab', 'al noor garden, faisalabad, punjab', 'al razzaq royals, sahiwal, punjab', 'al rehman garden, lahore, punjab', 'alfalah town, lahore, punjab', 'allama iqbal town, lahore, punjab', 'architects engineers housing society, lahore, punjab', 'askari 10, rawalpindi, punjab', 'askari 13, rawalpindi, punjab', 'askari 14, rawalpindi, punjab', 'askari 6, peshawar, khyber pakhtunkhwa', 'askari bypass, multan, punjab', 'askari, lahore, punjab', 'audit & accounts housing society, lahore, punjab', 'b-17, islamabad, islamabad capital', 'bahadurpur, multan, punjab', 'bahria nasheman, lahore, punjab', 'bahria orchard, lahore, punjab', 'bahria town rawalpindi, rawalpindi, punjab', 'bahria town, isla

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:00:50] "POST /predict_home_price HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:01:50] "GET / HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:01:51] "GET /app.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:01:51] "GET /app.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:01:52] "GET /get_location_names HTTP/1.1" 200 -


Locations loaded: ['abdalians cooperative housing society, lahore, punjab', 'adiala road, rawalpindi, punjab', 'afshan colony, rawalpindi, punjab', 'airport enclave, islamabad, islamabad capital', 'airport housing society, rawalpindi, punjab', 'airport road, lahore, punjab', 'al jalil garden, lahore, punjab', 'al noor garden, faisalabad, punjab', 'al razzaq royals, sahiwal, punjab', 'al rehman garden, lahore, punjab', 'alfalah town, lahore, punjab', 'allama iqbal town, lahore, punjab', 'architects engineers housing society, lahore, punjab', 'askari 10, rawalpindi, punjab', 'askari 13, rawalpindi, punjab', 'askari 14, rawalpindi, punjab', 'askari 6, peshawar, khyber pakhtunkhwa', 'askari bypass, multan, punjab', 'askari, lahore, punjab', 'audit & accounts housing society, lahore, punjab', 'b-17, islamabad, islamabad capital', 'bahadurpur, multan, punjab', 'bahria nasheman, lahore, punjab', 'bahria orchard, lahore, punjab', 'bahria town rawalpindi, rawalpindi, punjab', 'bahria town, isla

INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:54:26] "GET / HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:54:27] "GET /app.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:54:27] "GET /app.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 05:54:28] "GET /get_location_names HTTP/1.1" 200 -


Locations loaded: ['abdalians cooperative housing society, lahore, punjab', 'adiala road, rawalpindi, punjab', 'afshan colony, rawalpindi, punjab', 'airport enclave, islamabad, islamabad capital', 'airport housing society, rawalpindi, punjab', 'airport road, lahore, punjab', 'al jalil garden, lahore, punjab', 'al noor garden, faisalabad, punjab', 'al razzaq royals, sahiwal, punjab', 'al rehman garden, lahore, punjab', 'alfalah town, lahore, punjab', 'allama iqbal town, lahore, punjab', 'architects engineers housing society, lahore, punjab', 'askari 10, rawalpindi, punjab', 'askari 13, rawalpindi, punjab', 'askari 14, rawalpindi, punjab', 'askari 6, peshawar, khyber pakhtunkhwa', 'askari bypass, multan, punjab', 'askari, lahore, punjab', 'audit & accounts housing society, lahore, punjab', 'b-17, islamabad, islamabad capital', 'bahadurpur, multan, punjab', 'bahria nasheman, lahore, punjab', 'bahria orchard, lahore, punjab', 'bahria town rawalpindi, rawalpindi, punjab', 'bahria town, isla

ERROR:__main__:Exception on /predict_home_price [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-54-87f551445494>", line 38, in predict_home_price
    'estimated_price': predict_price(location